# 🍅 Detecção de Tomates com HDF5
Este notebook realiza:
- Leitura e extração de dados em `.h5`
- Análise exploratória com estatísticas e visualizações
- Treinamento de modelo simples de aprendizado de máquina


In [ ]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from glob import glob
from PIL import Image

from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# Altere para o caminho onde estão seus .h5 extraídos dos .tar
h5_folder = "./dados_h5_extraidos"
h5_files = sorted(glob(os.path.join(h5_folder, "*.h5")))

def carregar_dados_h5(caminho):
    with h5py.File(caminho, "r") as f:
        # Tente detectar as chaves corretas
        keys = list(f.keys())
        print("Chaves encontradas:", keys)
        imagem = np.array(f[keys[0]])  # geralmente RGB ou semântica
        return imagem

# Exemplo: carregar e mostrar uma imagem
imagem_exemplo = carregar_dados_h5(h5_files[0])
plt.imshow(imagem_exemplo)
plt.title("Exemplo de Imagem .h5")
plt.show()

In [ ]:
# Simular extração de features: média, std, etc
dados_features = []
for fpath in h5_files[:100]:  # usar só os primeiros 100 para teste rápido
    img = carregar_dados_h5(fpath)
    if img.ndim == 3:
        r_mean = np.mean(img[:, :, 0])
        g_mean = np.mean(img[:, :, 1])
        b_mean = np.mean(img[:, :, 2])
        r_std = np.std(img[:, :, 0])
        g_std = np.std(img[:, :, 1])
        b_std = np.std(img[:, :, 2])
        tomate_ratio = np.sum(img[:, :, 1] > 150) / img.size  # exemplo
        dados_features.append([r_mean, g_mean, b_mean, r_std, g_std, b_std, tomate_ratio])

df = pd.DataFrame(dados_features, columns=["r_mean", "g_mean", "b_mean", "r_std", "g_std", "b_std", "tomate_ratio"])
df.head()

In [ ]:
sns.pairplot(df)
plt.suptitle("Visualização das Features", y=1.02)
plt.show()

plt.figure(figsize=(12, 5))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
plt.title("Correlação entre Atributos")
plt.show()

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

kmeans = KMeans(n_clusters=2, random_state=42)
df['cluster'] = kmeans.fit_predict(X_scaled)

sns.scatterplot(data=df, x='r_mean', y='g_mean', hue='cluster', palette='viridis')
plt.title("Clusterização das Imagens por Cores")
plt.show()

In [ ]:
# Simulando um target binário com base no cluster
X = df.drop(columns=['cluster'])
y = df['cluster']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
print("Hiperparâmetros usados no RandomForest:")
print(clf.get_params())